# <center>  Data Scientist Technical Challenge <br> Vladislav Ryzhikh </center>

### Executive Summary

In this study, we aimed to optimise the effectiveness of a promotional campaign targeting potential customers for savings accounts. We conducted a comprehensive analysis that involved exploratory data analysis, feature engineering, class balancing, and feature selection. The Extra Trees Classifier emerged as the best model, achieving a precision of 0.331 on the test set. Through a tailored algorithm, we maximised the expected number of successful promotion calls while adhering to the constraints of the promotion blitz. The optimised policy is projected to increase overall profit by $66,320.63.

To further enhance our model's performance, we recommend incorporating multi-year data and refining the granularity of our dataset with specific time-of-call information. This would enable us to determine the optimal moments for contacting customers based on their preferences and availability. Additionally, access to data on all bank customers, including those not contacted, would allow us to formulate the problem as an Uplift problem, leading to a more refined approach.

Exploring other classification models like CatBoost and LightGBM, along with more sophisticated feature engineering and rigorous model evaluation techniques, could yield better results. Analysing the most important features for the model using explainability tools such as SHAP or LIME would provide valuable insights into the factors driving successful customer interactions.

In conclusion, our comprehensive data-driven approach has proven effective in enhancing the profitability of the promotional campaign. By continuously refining our methodology and incorporating additional data and techniques, we can further optimise our promotional efforts and maximise return on investment.

Written in Python 3.11.2 in VSCode. Requires Python 3.10+ <br>
To run the notebook, install the required packages using the following command:

```bash
pip install -r requirements.txt
```

And provide the path to the data folder in the `DATA_PATH` variable in the next cell.

In [1]:
DATA_PATH = "dataset.csv"

In [2]:
# standard libraries
from collections import Counter

# 3rd party
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from category_encoders import LeaveOneOutEncoder
from imblearn.over_sampling import SMOTENC
from sklearn.feature_selection import RFE

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import confusion_matrix, precision_score, roc_curve, auc

from ortools.linear_solver import pywraplp


In [3]:
# To see all outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

As the dataset is small (<5 Mb), we will use `pandas` as our main data analysis and manipulation tool. No need for `Polars` or `PySpark`.

In [4]:
df = pd.read_csv(DATA_PATH)

In [5]:
unique_person_characteristics = ["age", "job", "marital_status", "education", "default", "housing", "loan"]
n_groups = df.groupby(unique_person_characteristics).ngroups
print(f"{n_groups} ({n_groups / len(df) * 100:.2f}%) unique combinations of columns with person characteristics")

11535 (35.01%) unique combinations of columns with person characteristics


### General Assumptions

Despite the presence of only 11,535 unique combinations of personal characteristics in the dataset, we must assume that each entry represents a distinct customer. This assumption is necessary due to the lack of sufficient information to determine which calls were made to the same individual. Additionally, the time of the call is not specific enough to accurately trace back to any previous calls, further limiting our ability to identify repeat customers. As a result, treating each entry as a unique customer is the most reasonable approach under the current data constraints.

Furthermore, based on the column descriptions provided, we will assume that each previous call was about other marketing campaigns. Consequently, it is reasonable to assume that a customer can be contacted regarding the opening of a high-interest savings account with us only once. This assumption aligns with the available data and helps simplify the analysis.

In accordance with the given instructions, we will assume that the data has been collected from a random sample of customers. However, we must make an even stronger assumption: we will assume that the calls were made on a random basis as well. This assumption is crucial to ensure that the data is representative of the entire customer population (all customers of GloboMegaBank). If the calls were made in a non-random manner, the data would not accurately represent the population, and the results of our analysis would be invalid. Given the absence of information about all the customers of the client's bank, including those who have not been contacted at all, this assumption is necessary for us to draw inferences about the customer population from the sample data.

### Exploratory analysis

In [6]:
df.head()
df.info()

,age,job,marital_status,education,default,housing,loan,contact,month,day_of_week,days_since_last_contact,n_previous_contacts,previous_outcome,success
0,49,blue_collar,married,9y,unknown,no,no,cellular,nov,wed,999,0,nonexistent,False
1,37,entrepreneur,married,university_degree,no,no,no,telephone,nov,wed,999,1,failure,False
2,78,retired,married,4y,no,no,no,cellular,jul,mon,999,0,nonexistent,True
3,36,admin,married,university_degree,no,yes,no,telephone,may,mon,999,0,nonexistent,False
4,59,retired,divorced,university_degree,no,no,no,cellular,jun,tue,999,0,nonexistent,False


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32950 entries, 0 to 32949
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   age                      32950 non-null  int64 
 1   job                      32950 non-null  object
 2   marital_status           32950 non-null  object
 3   education                32950 non-null  object
 4   default                  32950 non-null  object
 5   housing                  32950 non-null  object
 6   loan                     32950 non-null  object
 7   contact                  32950 non-null  object
 8   month                    32950 non-null  object
 9   day_of_week              32950 non-null  object
 10  days_since_last_contact  32950 non-null  int64 
 11  n_previous_contacts      32950 non-null  int64 
 12  previous_outcome         32950 non-null  object
 13  success                  32950 non-null  bool  
dtypes: bool(1), int64(3), object(10)
memor

In [7]:
num_columns = df.select_dtypes(include=["int"]).columns
cat_columns = df.select_dtypes(include=["object"]).columns

# Convert ordered categorical columns to pd.Categorical
df['month'] = pd.Categorical(
    df['month'],
    categories=['mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'],
    ordered=True
)
df['day_of_week'] = pd.Categorical(
    df['day_of_week'],
    categories=['mon', 'tue', 'wed', 'thu', 'fri'],
    ordered=True
)
# For now we will consider unknown as the lowest education level; We will encode `unknown` during feature engineering
# We will rank professional_course as the second highest education level just below university_degree
df['education'] = pd.Categorical(
    df['education'],
    categories=['unknown', 'illiterate', '4y', '6y', '9y', 'high_school', 'professional_course', 'university_degree'], 
    ordered=True
)

We will exclude customers, who have not been contacted previously (`days_since_last_contact` == 999) from visualisation to improve the readability of the `days_since_last_contact` distribution.

In [8]:
def create_num_histogram(col: str, success: bool) -> go.Histogram:
    """Create a histogram for a numerical column.
    
    Excludes the 999 value for days_since_last_contact to
    get a better view of the distribution.

    `age` is the first column. Visible on initialisation.

    Args:
        col (str): The column name.
        success (bool): Whether to count successful or unsuccessful calls.
    
    Returns:
        go.Histogram: The histogram.

    """

    col_display = df[df["success"] == success][col] \
        if col != "days_since_last_contact" \
        else df[(df["days_since_last_contact"] != 999) & (df["success"] == success)][col]
    
    return go.Histogram(
        x=col_display,
        nbinsx=20,
        name="Success" if success else "Failure",
        marker=dict(color='rgba(191, 227, 180, 0.9)' if success else 'rgba(244, 113, 116, 0.9)'),
        legendgroup="Success" if success else "Failure",
        showlegend=True,
        visible=col == "age",  # only show the job column on initialisation
    )

num_histograms = [create_num_histogram(col, success) for col in num_columns for success in [False, True]]

buttons = [
    dict(
        label=col, 
        method="update", 
        args=[
            {"visible": [col == c for c in num_columns for success in [False, True]]},
            {
                "title": f"Numerical Features Histograms - {col.capitalize()}",  # update the title with the selected column name
                "xaxis": {"title": col}  # update the x-axis title with the selected column name
            },
        ]
    ) for col in num_columns
]

layout = go.Layout(
    title="Numerical Features Histograms - Age",  # initial title
    barmode="stack",  # stack hists
    updatemenus=[
        dict(
            type="buttons",
            showactive=True,
            buttons=buttons,
        )
    ],
    xaxis_title="age",  # initial x-axis title
    yaxis_title="Count",  # static y-axis title
)

fig = go.Figure(data=num_histograms, layout=layout)
fig.show()

# statistics on customers who have not been contacted previously (days_since_last_contact == 999)
not_contacted = (df["days_since_last_contact"] == 999).sum()
print(f"Number of customers who have not been contacted previously: {not_contacted} ({not_contacted / len(df) * 100:.2f}%)")
print(f"Success rate of customers who have not been contacted previously: "
      f"{df[df['days_since_last_contact'] == 999]['success'].value_counts(normalize=True)[True] * 100:.2f}%")

Number of customers who have not been contacted previously: 31724 (96.28%)
Success rate of customers who have not been contacted previously: 9.24%


In [9]:
def create_cat_bar(col: str, success: bool) -> tuple[go.Bar, list]:
    """Create a bar chart for a categorical column.

    By default, columns are sorted in descending order
    of the number of observations.
    
    For `month` and `day_of_week`, the order is changed to
    chronological order.
    
    For `education`, the order is changed to descending
    order of education level.

    `job` is the first column. Visible on initialisation.

    Saves the order of categories shown in a list for later use 
    in other visualisations.
    
    Args:
        col (str): The column name.
        success (bool): Whether to count successful or unsuccessful calls.
    
    Returns:
        go.Bar: The bar chart.
        list: The order of categories shown.

    """

    col_display = df[df["success"] == success][col]
    
    if col == 'month':
        months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
        ordered_categories = sorted(col_display.unique(), key=lambda x: months.index(x))
    
    elif col == 'day_of_week':
        days = ['mon', 'tue', 'wed', 'thu', 'fri']
        ordered_categories = sorted(col_display.unique(), key=lambda x: days.index(x))

    elif col == 'education':
        education = ['unknown', 'illiterate', '4y', '6y', '9y', 'high_school', 'professional_course', 'university_degree']
        ordered_categories = sorted(col_display.unique(), key=lambda x: - education.index(x))
    
    else:
        ordered_categories = sorted(col_display.unique(), key=lambda x: df[col].value_counts()[x], reverse=True)

    fig = go.Bar(
        x=ordered_categories,
        y=[col_display.value_counts()[cat] for cat in ordered_categories],
        name="Success" if success else "Failure",
        marker=dict(color='rgba(191, 227, 180, 0.9)' if success else 'rgba(244, 113, 116, 0.9)'),
        legendgroup="Success" if success else "Failure",
        showlegend=True,
        visible=col == "job"  # only show the job column on initialisation
    )

    return fig, ordered_categories

cat_bars = []
orders = {}  # dict[column_name, list[order]] save the order of categories shown for later use in other visualisations
for col in cat_columns:
    for success in [False, True]:
        cat_bar, order = create_cat_bar(col, success)
        cat_bars.append(cat_bar)
        orders[col] = order

buttons = [
    dict(
        label=col, 
        method="update",
        args=[
            {"visible": [col == c for c in cat_columns for _ in [False, True]]},
            {
                "title": f"Categorical Features Bar Charts - {col.capitalize()}",  # update the title with the selected column name
                "xaxis": {"title": col},  # update the x-axis title with the selected column name
            }
        ]
    ) for col in cat_columns
]

layout = go.Layout(
    title="Categorical Features Bar Charts - Job",  # initial title
    barmode="stack",  # stack the bar charts
    updatemenus=[
        dict(
            type="buttons",
            showactive=True,
            buttons=buttons,
        ),
    ],
    xaxis_title="job",  # initial x-axis title
    yaxis=dict(
        title="Count",  # static y-axis title
        title_standoff=0,
    )
)

fig = go.Figure(data=cat_bars, layout=layout)
fig.show()

In [10]:
average_success_rate = df.success.value_counts()[1] / len(df)
print(f"Average success rate: {average_success_rate * 100:.2f}%")

Average success rate: 11.27%


In [11]:
df.groupby("contact")["success"].mean()

contact
cellular     0.147025
telephone    0.052981
Name: success, dtype: float64

The target column (`success`) exhibits a significant imbalance, with a success rate of just 11.27%. We will need to employ balancing techniques to mitigate bias during model training.

The `default` positive class is notably underrepresented, with only 3 positive "yes" values. The observed default distribution is likely to reflect the population distribution; therefore, we will not utilise any sampling techniques to balance the feature. Additionally, due to the considerable underrepresentation of this class, we will convert positive default observations to "unknown" in order to preserve as much information as possible while minimising the risk of overfitting.

The number of calls to cellphones is nearly double the number of calls to landlines, which is expected given the prevalence of cellphones over landlines. The success rate of calls to cellphones is three times higher than that of calls to landlines.

The dataset covers a 10-month period from March to December, with the number of customer contacts per month varying considerably, peaking in May and reaching its lowest point in December. Meanwhile, all calls were made during weekdays. The day of week (`day_of_week`) histogram is fairly consistent, with 6400 - 6800 contacts and a 10% - 12% average success rate per day of week.

Let's examine the success rate more closely.

In [12]:
# Show only last output
InteractiveShell.ast_node_interactivity = "last"

In [13]:
def create_scatters(col: str, orders_: dict) -> tuple[go.Scatter, go.Scatter]:
    """Create scatter plots for a dual-line graph.

    The order of the categories  of categorical columns
    is the same as in the previous bar charts.
    
    Excludes the 999 value for days_since_last_contact to
    get a better view of the dependency.

    Args:
        col (str): The column name.
        orders_ (dict): The order of categories shown in the bar charts.
    
    Returns:
        go.Scatter: Success rate scatter plot.
        go.Scatter: Count scatter plot.
    
    """

    grouped = df.groupby(col).agg({'success': ['mean', 'count']}).reset_index()
    grouped.columns = [col, 'success_rate', 'count']

    if col == 'days_since_last_contact':
        grouped = grouped[grouped[col] != 999]
        

    # reorder rows based on order in the bar charts
    if col in cat_columns:
        grouped = grouped.set_index(col).reindex(orders_[col]).reset_index()

    return (
        go.Scatter(x=grouped[col], y=grouped['success_rate'], name='Success Rate', visible=col == 'month'),
        go.Scatter(x=grouped[col], y=grouped['count'], name='Number of Calls', visible=col == 'month'),
    )

# select columns to plot
cols = ["month", "day_of_week", "education", "age", "days_since_last_contact", "n_previous_contacts"]

# `orders` is a dictionary created during making the categorical bar charts.
success_rates, counts = zip(*[create_scatters(col, orders) for col in cols])

# Create a subplot
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add all the success rate and count scatters to the subplot
for sr, cnt in zip(success_rates, counts):
    fig.add_trace(sr, secondary_y=False)
    fig.add_trace(cnt, secondary_y=True)

# Add a horizontal line indicating average success rate
# Average success rate was calculated above
fig.add_hline(
    y=average_success_rate, 
    line_color='blue',
    annotation_text="Mean Success Rate",
    annotation_position="top left",
    line_width=1
)

buttons = [
    dict(
        label=col,
        method="update",
        args=[
            {
                "visible": [i == idx for i in range(len(success_rates)) for _ in range(2)],
            },
            {
                "xaxis.title.text": col,
                "title": f"Success Rate and Number of Calls - {col.capitalize()}",  # update the title with the selected column name
            },
        ],
    ) for idx, col in enumerate(cols)
]

fig.update_layout(
    title="Success Rate and Number of Calls - Month",  # initial title
    updatemenus=[
        {
            "type": "buttons",
            "showactive": True,
            "buttons": buttons,
        },
    ],
    xaxis_title="month",  # initial x-axis title
    yaxis=dict(
        title="Success Rate",  # static y-axis title
        title_standoff=0,
    ),
    yaxis2=dict(
        title='Number of Calls',  # static secondary y-axis title
        overlaying='y',
        side='right'
    ),
)

fig.show()

In [14]:
grouped = df.groupby('job').agg({'success': ['mean', 'count']}).reset_index()
grouped.columns = ['job', 'success_rate', 'count']
fig = px.scatter(grouped, x="job", y="success_rate", size="count")
fig.add_hline(
    y=average_success_rate, 
    line_color='blue',
    annotation_text="Mean Success Rate",
    annotation_position="top left",
    line_width=1
)
fig.update_layout(
    title="Success Rate by Job",
    xaxis_title="Job",
    yaxis_title="Success Rate",
)

fig.show()

The dual-line graph above demonstrates a negative correlation between the success rate and the number of calls made in a given month. This observation suggests that during months with fewer calls, the calls tend to be more targeted and precise. In contrast, during high-volume call campaigns, such as in May, which recorded the highest number of customer contacts, a larger proportion of cold calls were made, resulting in the lowest overall success rate.

The dual-line graph illustrating the relationship between the day of the week of contact (`day_of_week`) and the success rate supports the observation mentioned above. Although the success rate is higher during the middle of the week (Tuesday, Wednesday, Thursday), the difference is not substantial.

The dual-line graph comparing the number of previous contacts (`n_previous_contacts`) and the success rate indicates that customers who are contacted more frequently are more likely to accept the offer, regardless of the previous outcome (`previous_outcome`). It should be noted that the success rate drops significantly for customers contacted seven times. However, this observation can be disregarded due to underrepresentation, as only one customer was contacted seven times.

Examining the dual-line graph of the number of days since the last contact (`days_since_last_contact`) and the success rate, and excluding observations for customers who have not been contacted after 18 days due to underrepresentation, we can observe that customers are more likely to accept the offer than decline it (success rate > 50%) if they were contacted either on the same day or between 2 and 16 days.

Regarding the age column, the success rate is above the mean for customers younger than 30 and older than 59. This observation suggests that younger customers are more likely to accept the offer, possibly because they might not have a high-interest savings account yet, and our offer could be a valuable opportunity for them. Meanwhile, older customers (>59) are also generally more interested in our offer. In contrast, the success rate for customers aged between 30 and 59, who are our target clients, is below the mean. This finding implies that customers in their 30s, 40s, and 50s are less likely to accept the offer, possibly because they already have a savings account.

Interestingly, the success rate based on education level exhibits a "U" shape when arranged in descending order of education level. The most and least educated customers are more likely to accept the offer.

The success rate for retired professionals and students is twice and nearly three times the average success rate, respectively. This observation indicates that retired professionals and students are more inclined to accept the offer compared to other customers. However, these groups are among the least represented in the dataset, with only 1,366 and 711 observations, respectively. Conversely, blue-collar and service workers exhibit the lowest likelihood of accepting the offer. Administrative professionals, on the other hand, are well-represented in the dataset and demonstrate a success rate above the mean, making them appealing targets for calling.

In [15]:
df[(df.days_since_last_contact == 999) & (df.n_previous_contacts > 0)]["previous_outcome"].value_counts()

previous_outcome
failure    3308
Name: count, dtype: int64

From the task and column descriptions we know that the above is not possible. `days_since_last_contact` is 999 for customers, who have not been contacted before. Hence, the `n_previous_calls` must be 0 for these customers. We will drop these rows from the dataset.

In [16]:
df = df[~((df.days_since_last_contact == 999) & (df.n_previous_contacts > 0))]

Since the company wants to ensure that the promotion is offered fairly to all members of the community, regardless of demographic or socioeconomic characteristics, let's see whether the promotion had been offered fairly.

In [17]:
AGE_THRESHOLD = 40
PROFESSIONS = ['blue collar', 'housemaid', 'admin', 'student', 'technician', 'unemployed']
TRAIN_MONTHS = ["mar", "apr", "may", "jun", "jul", "aug", "sep"]
TEST_MONTHS = ["oct", "nov", "dec"]

In [18]:
data = {
    "< 40 y.o.": [
        (df[df.month.isin(TRAIN_MONTHS)].age < AGE_THRESHOLD).sum() / df.month.isin(TRAIN_MONTHS).sum(),
        (df[df.month.isin(TEST_MONTHS)].age < AGE_THRESHOLD).sum() / df.month.isin(TEST_MONTHS).sum(),
        (df.age < AGE_THRESHOLD).sum() / len(df),
    ],
    "Profession": [
        (df[df.month.isin(TRAIN_MONTHS)].job.isin(PROFESSIONS)).sum() / df.month.isin(TRAIN_MONTHS).sum(),
        (df[df.month.isin(TEST_MONTHS)].job.isin(PROFESSIONS)).sum() / df.month.isin(TEST_MONTHS).sum(),
        (df.job.isin(PROFESSIONS)).sum() / len(df),
    ],
}

index = [
    "Mar - Sep",
    "Oct - Dec",
    "Total",
]

df_constraints = pd.DataFrame(data, index=index)
df_constraints *= 100
df_constraints.style.format("{:.2f}%")

,< 40 y.o.,Profession
Mar - Sep,54.71%,48.70%
Oct - Dec,51.79%,50.76%
Total,54.40%,48.93%


The table above illustrates that during the March to September period, the second constraint was not met. However, it was satisfied during the last three months of the year when the data was collected. Furthermore, both constraints were fulfilled in the final quarter, indicating that the company is making progress in ensuring that the promotion is offered fairly to all members of the community, regardless of demographic or socioeconomic characteristics.

### Feature engineering

We will create a bunch of new features and then allow feature selection methods to determine which features to include in the final model. Feature selection methods will be employed later in our analysis. We could use Principal Component Analysis (PCA) to reduce the dimensionality of the dataset; however, this would compromise the interpretability of the model.

We will encode the `education` column as an ordinal categorical variable, ranking from 0 to 6, representing the education level from lowest to highest. We will introduce a new column, `unknown_education`, to indicate if the education level is unknown. This approach enables the models to learn from this information while encoding the unknown education level as the mean of the other education levels. We will apply a similar strategy for the `housing` and `loan` columns.

As previously mentioned, we will convert positive `default` observations to "unknown" to retain as much information as possible while minimising the risk of overfitting. We will then convert the `default` column into a binary variable.

Additionally, we will convert the `contact` column into a binary variable, using cellular as the positive class.

Building on our earlier discussions, we will extract information about the optimal contact time in days from the `days_since_last_contact` column. We will create a new column, `optimal_contact_period`, to indicate whether the contact was made during the optimal period (less than 18 days). Furthermore, we will introduce a new column, `was_contacted_before` (`n_previous_contacts` > 0), to denote whether the customer has been contacted previously.

We will create `is_professional` and `age_threshold` columns for the two groups that the bank is particularly interested in.

Lastly, we will generate an `age_group` column to categorize customers into age groups and an `age_target` column for the target age audience (30 - 59 years old).

In [19]:
# Education
education_mapping = {
    "unknown": -1,
    "illiterate": 0,
    "4y": 1,
    "6y": 2,
    "9y": 3,
    "high_school": 4,
    "professional_course": 5,
    "university_degree": 6,
}
df.education = df.education.astype(str)
df.education = df.education.map(education_mapping)
df["unknown_education"] = (df.education == -1).astype(int)
df.loc[df.education == -1, "education"] = df.loc[df.education != -1, "education"].mean()

# Contact
df.contact = df.contact.map({"cellular": 1, "telephone": 0})

# Housing
df.housing = df.housing.map({"yes": 1, "no": 0, "unknown": -1})
df["unknown_housing"] = (df.housing == -1).astype(int)
df.loc[df.housing == -1, "housing"] = df.loc[df.housing != -1, "housing"].mean()

# Loan
df.loan = df.loan.map({"yes": 1, "no": 0, "unknown": -1})
df["unknown_loan"] = (df.loan == -1).astype(int)
df.loc[df.loan == -1, "loan"] = df.loc[df.loan != -1, "loan"].mean()

# Default
df.default = df.default.map({"yes": 1, "unknown": 1, "no": 0})

# Days since last contact
df["optimal_contact_period"] = (df.days_since_last_contact < 17).astype(int)

# Job
df["is_profession"] = df.job.isin(PROFESSIONS).astype(int)

# Age
df["age_group"] = pd.cut(df.age, bins=[0, 20, 30, 40, 50, 60, 70, 80, 90, 100], labels=False).astype(int)
df["age_threshold"] = (df.age < AGE_THRESHOLD).astype(int)
df["age_target"] = ((df.age >= 30) | (df.age <= 59)).astype(int)

# Number of previous contacts
df["was_contacted_before"] = (df.n_previous_contacts > 0).astype(int)

Target Encoding: The following section of code has been commented out as it was found that target encoding negatively impacted the performance of the model. It seems that the application of target encoding introduced a strong bias towards the training set, resulting in overfitting. We experimented with Leave One Out Encoding (LOO) on the `job`, `marital_status`, `day_of_week`, `previous_outcome`, and `age_group` columns, but ultimately decided against using this method due to the issues encountered.

cols_to_target_encode = [`job`, `marital_status`, `day_of_week`, `previous_outcome`, `age_group`]
target_encoded_col_names = [col + "_target_encoded" for col in cols_to_target_encode]

df["age_group"] = df["age_group"].astype(str)

loo = LeaveOneOutEncoder()

is_train = df.month.isin(TRAIN_MONTHS)
X_train = df[is_train]
X_test = df[~is_train]

y_train = X_train.pop("success")
y_test = X_test.pop("success")

X_train_encoded = loo.fit_transform(X_train[cols_to_target_encode], y_train)
X_test_encoded = loo.transform(X_test[cols_to_target_encode])

X_train_encoded.columns = target_encoded_col_names
X_test_encoded.columns = target_encoded_col_names
X_train = pd.concat([X_train, X_train_encoded], axis=1)
X_test = pd.concat([X_test, X_test_encoded], axis=1)

X_train["age_group"] = X_train["age_group"].astype(int)
X_test["age_group"] = X_test["age_group"].astype(int)

In [20]:
# Split on train and test
is_train = df.month.isin(TRAIN_MONTHS)
X_train = df[is_train]
X_test = df[~is_train]

y_train = X_train.pop("success")
y_test = X_test.pop("success")

# save for part 2
X_test_job = X_test.job.isin(PROFESSIONS).astype(int)
X_test_age = (X_test.age < AGE_THRESHOLD).astype(int)

In [21]:
# Additional dummies

# Job; marital status; day of week; previous outcome
cols_to_one_hot_encode = ["job", "marital_status", "day_of_week", "previous_outcome"]
X_train = pd.get_dummies(X_train, columns=cols_to_one_hot_encode, drop_first=False)
X_test = pd.get_dummies(X_test, columns=cols_to_one_hot_encode, drop_first=False)

In [22]:
# drop redundant and original categorical columns
cols_to_drop = ["month", "days_since_last_contact"]
X_train = X_train.drop(cols_to_drop, axis=1)
X_test = X_test.drop(cols_to_drop, axis=1)

In [23]:
# Confirm order of columns just in case
(X_train.columns == X_test.columns).all()

True

### Class Balancing

We will use SMOTENC to balance the target class. SMOTENC is an extension of SMOTE that can handle categorical variables.

In [24]:
categorical_features = {
    'education', 'default', 'housing', 'loan', 'contact', 'n_previous_contacts',
    'unknown_education', 'unknown_housing', 'unknown_loan', 'optimal_contact_period',
    'is_profession', 'age_threshold', 'was_contacted_before',
    'job_admin', 'job_blue_collar', 'job_entrepreneur', 'job_housemaid', 'job_management',
    'job_retired', 'job_self_employed', 'job_services', 'job_student', 'job_technician',
    'job_unemployed', 'job_unknown', 'marital_status_divorced', 'marital_status_married',
    'marital_status_single', 'marital_status_unknown', 'day_of_week_mon', 'day_of_week_tue',
    'day_of_week_wed', 'day_of_week_thu', 'day_of_week_fri', 'previous_outcome_failure',
    'previous_outcome_nonexistent', 'previous_outcome_success',
}
categorical_features_mask = [col in categorical_features for col in X_train.columns]
smote = SMOTENC(random_state=42, categorical_features=categorical_features_mask)
X_train, y_train = smote.fit_resample(X_train, y_train)

### Feature Selection

The X_train dataset contains 45 features, and through experimentation, we have determined that selecting 25 features yields best results. We will utilise Recursive Feature Elimination (RFE) in conjunction with the Random Forest Classifier for this task. It is important to note that this approach may introduce some bias towards the Random Forest Classifier; however, it serves as a solid foundation from which we can refine our feature selection process. Other techniques, such as mRMR can be used to select features; however, we will not be exploring these techniques in this notebook.

In [25]:
estimator = RandomForestClassifier(random_state=42)

selector = RFE(estimator, n_features_to_select=25, step=1)
selector = selector.fit(X_train, y_train)
selected_features = selector.support_

X_train = X_train.loc[:, selected_features]
X_test = X_test.loc[:, selected_features]

X_train.columns

Index(['age', 'education', 'default', 'housing', 'loan', 'contact',
       'n_previous_contacts', 'optimal_contact_period', 'is_profession',
       'age_group', 'age_threshold', 'was_contacted_before', 'job_admin',
       'job_blue_collar', 'job_technician', 'marital_status_divorced',
       'marital_status_married', 'marital_status_single', 'day_of_week_mon',
       'day_of_week_tue', 'day_of_week_wed', 'day_of_week_thu',
       'day_of_week_fri', 'previous_outcome_nonexistent',
       'previous_outcome_success'],
      dtype='object')

### Machine Learning
For this problem, we have chosen to use `XGBoost` as our primary model, as it [has proven](https://www.kdnuggets.com/2015/12/harasymiv-lessons-kaggle-machine-learning.html) to be a very powerful algorithm for tabular data. To optimise the model, we will employ GridSearchCV to identify the best hyperparameters, while utilising StratifiedKFold for cross-validation in order to balance the target class distribution. We will also experiment with `ExtraTreesClassifier`.

Considering the limited capacity of the customer support team, we will prioritise precision as our scoring metric to minimise the likelihood of unsuccessful calls. If the company had a larger customer support team and aimed to reach as many customers as possible with the promotion, recall would be a more appropriate scoring metric.

In [26]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [27]:
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'n_estimators': [100, 200, 300],
}

xgb_clf = XGBClassifier(random_state=42)

grid_search_xgb = GridSearchCV(
    xgb_clf,
    param_grid,
    scoring='precision',
    cv=cv,
    n_jobs=-1,
    verbose=1,
)
grid_search_xgb.fit(X_train, y_train)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_typ...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=42, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [100, 200, 300]},
             scoring='precision', verbose=1)

In [ ]:
param_grid = {
    'n_estimators': [10, 50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

xt_clf = ExtraTreesClassifier(random_state=42)

grid_search_xt = GridSearchCV(
    xt_clf,
    param_grid,
    scoring='precision',
    cv=cv,
    n_jobs=-1,
    verbose=1,
)
grid_search_xt.fit(X_train, y_train)


In [29]:
print(f"XGBoost best score: {grid_search_xgb.best_score_:.3f}")
print(f"ExtraTrees best score: {grid_search_xt.best_score_:.3f}")

XGBoost best score: 0.864
ExtraTrees best score: 0.897


Extra Trees has shown a better performance than XGBoost in terms of precision on train set. Since is probably since we've used Random Forest as a feature selector, as since the two models are similar in nature, it makes sense that the Extra Trees model would perform better.

In [30]:
# train best estimator on full train set
xt = grid_search_xt.best_estimator_.fit(X_train, y_train)

In [31]:
# roc curve
y_prob = xt.predict_proba(X_test)[:, 1]
fpr, tpr, _ = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)

fig = px.area(
    x=fpr, y=tpr,
    title=f'ROC Curve (AUC = {roc_auc:.2f})',
    labels=dict(x='False Positive Rate', y='True Positive Rate'),
)
fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1
)

fig.show()

From the roc curve above we see that although the performance of the model is not ideal, it is better than random guessing.

In [32]:
y_pred = xt.predict(X_test)
cm = confusion_matrix(y_test, y_pred)

cm_df = pd.DataFrame(cm, columns=['Negative', 'Positive'], 
                     index=['Negative', 'Positive'])

fig = px.imshow(cm_df, color_continuous_scale='Blues')

fig.update_xaxes(title='Predicted Class', side='top')
fig.update_yaxes(title='Actual Class', autorange="reversed")
fig.update_layout(title='Confusion Matrix')

for i, row in enumerate(cm):
    for j, value in enumerate(row):
        fig.add_annotation(
            go.layout.Annotation(
                text=str(value),
                x=j,
                y=i,
                showarrow=False,
                font=dict(
                    color='black',
                    size=16
                )
            )
        )


fig.show()

In [33]:
print(f"Precision score on test set: {precision_score(y_test, y_pred):.3f}")

Precision score on test set: 0.331


### Part 2: Optimisation

We will formulate the problem as a Mixed Integer Linear Programming (MILP) problem. We will use Google's OR-Tools as a solver.

In [34]:
# Obtain predictions given the day of the call
X_test_day = X_test.copy()
predictions = []

for i in range(5):
    
    X_test_day['day_of_week_mon'] = int(0 == i)
    X_test_day['day_of_week_tue'] = int(1 == i)
    X_test_day['day_of_week_wed'] = int(2 == i)
    X_test_day['day_of_week_thu'] = int(3 == i)
    X_test_day['day_of_week_fri'] = int(4 == i)

    y_pred = xt.predict_proba(X_test_day)
    predictions.append(y_pred)

y_prob_mon, y_prob_tue, y_prob_wed, y_prob_thu, y_prob_fri = predictions
y_prob_mon = y_prob_mon[:, 1]
y_prob_tue = y_prob_tue[:, 1]
y_prob_wed = y_prob_wed[:, 1]
y_prob_thu = y_prob_thu[:, 1]
y_prob_fri = y_prob_fri[:, 1]

df_optimisation = pd.DataFrame(
    {
        "age": X_test_age,
        "job": X_test_job,
        "y_prob_mon": y_prob_mon,
        "y_prob_tue": y_prob_tue,
        "y_prob_wed": y_prob_wed,
        "y_prob_thu": y_prob_thu,
        "y_prob_fri": y_prob_fri,
    },
).reset_index(drop=True)


In [35]:
solver = pywraplp.Solver.CreateSolver('SCIP')

# definition of the varriables to solve for
num_customers = len(df_optimisation)
days = ['mon', 'tue', 'wed', 'thu', 'fri']
x = {}

for i in range(num_customers):
    for day in days:
        x[i, day] = solver.BoolVar(f'x[{i},{day}]')

# Objective
solver.Maximize(
    solver.Sum(
        x[i, day] * df_optimisation.at[i, f'y_prob_{day}'] 
        for i in range(num_customers) 
        for day in days
    )
)

# Constraints

# 1000 total calls
solver.Add(solver.Sum(x[i, day] for i in range(num_customers) for day in days) <= 1000)

# 400 calls per day
for day in days:
    solver.Add(
        solver.Sum(
            x[i, day] 
            for i in range(num_customers)
        ) <= 400
    )

# Total calls to customers with age < 40 is at least 50% of total calls
solver.Add(
    solver.Sum(
        x[i, day] * df_optimisation.at[i, 'age']
        for i in range(num_customers)
        for day in days
    ) >= 0.5 * solver.Sum(
        x[i, day]
        for i in range(num_customers)
        for day in days
    )
)
# Total calls to customers with a special job is at least 50% of total calls
solver.Add(
    solver.Sum(
        x[i, day] * df_optimisation.at[i, 'job'] 
        for i in range(num_customers) 
        for day in days
    ) >= 0.5 * solver.Sum(
        x[i, day] 
        for i in range(num_customers) 
        for day in days
    )
)

solver.Solve() == pywraplp.Solver.OPTIMAL

True

In [36]:
num_calls = len([x[i, day] for i in range(num_customers) for day in days if x[i, day].solution_value() > 0])
total_proba = solver.Objective().Value()
expected_revenue = total_proba * 100
expected_profit = expected_revenue - num_calls * 10
print(f"Optimised expected revenue: ${total_proba * 100:,.2f}")
print(f"Optimised expected profit: ${expected_profit:,.2f}")

Optimised expected revenue: $93,928.11
Optimised expected profit: $83,928.11


In [37]:
# Random guessing
n_iter = 1000

rng = np.random.default_rng(42)
guesses = 0
expected_revenues = []
while True: 
    rows = rng.choice(num_customers, 1000, replace=False)

    # make sure we meet the constraints
    if np.sum(df_optimisation.loc[rows, 'age'] < 40) < 500:
        continue
    if np.sum(df_optimisation.loc[rows, 'job'] == 1) < 500:
        continue
    
    # randomly allocate calls to days
    while True:
        daily_calls = rng.choice(days, 1000, replace=True)
        daily_call_counts = Counter(daily_calls)

        # make sure we meet the daily call constraint
        if all(count <= 400 for count in daily_call_counts.values()):
            break
    
    # calculate the expected revenue
    total_proba = 0
    for row, day in zip(rows, daily_calls):
        total_proba += df_optimisation.loc[row, f'y_prob_{day}']
    
    expected_revenues.append(total_proba * 100)

    guesses += 1

    if guesses == n_iter:
        break

expected_revenue = np.mean(expected_revenues)
expected_profit = expected_revenue - 10000

print(f"Randomised expected revenue: ${expected_revenue:,.2f}")
print(f"Randomised expected profit: ${expected_profit:,.2f}")

Randomised expected revenue: $27,607.48
Randomised expected profit: $17,607.48


The expected impact on the profit of the optimised policy relative to a randomised feasible allocation is $66,320.63.


### Conclusion

In this notebook, we conducted a comprehensive analysis of the given dataset through multiple stages. First, we performed exploratory data analysis to gain insights into the data and identify potential trends. Next, we carried out feature engineering to create new variables that could potentially enhance the performance of our predictive model.

To address class imbalance in the data, we applied suitable techniques to balance the classes, followed by feature selection to retain only the most relevant predictors. We then proceeded to train a classification model that aimed to predict the probability of a customer accepting the promotional offer. In order to maximise the likelihood of a successful call, we focused on optimising precision as our evaluation metric.

The Extra Trees Classifier emerged as the best model with a precision of 0.331 on the test set. Utilising the predicted probabilities generated by this model, we devised an algorithm to maximise the expected number of successful promotion calls, while adhering to the constraints specified for the promotion blitz.

As a result, the optimised policy is projected to increase the overall profit by $66,320.63 relative to randomised calling. This demonstrates the effectiveness of our approach in enhancing the profitability of the promotional campaign through a combination of data analysis, feature engineering, and optimisation techniques.

### Opportunities for further improvement  

Incorporating multi-year data and enhancing the granularity of our dataset with specific time-of-call information can significantly improve our model's performance. By identifying the most opportune moments to contact customers, we can account for their preferences and availability, thereby increasing the likelihood of successfully encouraging them to open savings accounts with us. Moreover, if we had access to data on all bank customers, including those not contacted, we could have formulated the problem as an Uplift problem, further refining our approach.

Exploring other classification models, such as CatBoost and LightGBM, could potentially yield better results. Further improvements can be made through more sophisticated feature engineering, more efficient model training, and more rigorous model evaluation techniques. Additionally, analysing the most important features for the model using explainability tools like SHAP or LIME could provide valuable insights into the factors driving successful customer interactions. By continually refining our methodology, we can optimise the effectiveness of our promotional efforts and maximise the return on investment.